# Exploring North Carolina NO<sub>2</sub> hourly emmisions

From the [EPA site on Nitrogen Dioxide pollution](https://www.epa.gov/no2-pollution), "NO<sub>2</sub> primarily gets in the air from the burning of fuel. NO<sub>2</sub> forms from emissions from cars, trucks and buses, power plants, and off-road equipment."

**Data Source:** [EPA pregenerated hourly data files](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Raw)

**I'm using hourly data so we can explore the data at many different time scales, and in doing so, learn how to do aggregations at many scales on dates and times.**

The EPA has hourly data for Ozone, SO<sub>2</sub>, CO, and NO<sub>2</sub> for all sites in the US. **Because that data is quite large, I have filtered down to only North Carolina sites**, and I excluded Lee County because that data was incomplete for 2018. I have also filtered down to a subset of the columns, leaving only the State, County, Site, Date, Time and Measured value.

Here is an example of the output if you load in one row from the original data. (Note that not all columns are visible in this output...)

<img src="images/EPA_data_head.png" />

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

## Load NC subset for exploration


In [1]:
import pandas as pd
import polars as pl
import polars.selectors as cs
import altair as alt

In [2]:
df = pl.read_csv("data/AirDataEPA/NC_NO2_hourly_2018.csv")
df.head()

Site Num,Date Local,Time Local,Sample Measurement,State Name,County Name
i64,str,str,f64,str,str
22,"""2018-01-01""","""00:00""",4.4,"""North Carolina""","""Forsyth"""
22,"""2018-01-01""","""01:00""",5.1,"""North Carolina""","""Forsyth"""
22,"""2018-01-01""","""02:00""",3.6,"""North Carolina""","""Forsyth"""
22,"""2018-01-01""","""03:00""",4.1,"""North Carolina""","""Forsyth"""
22,"""2018-01-01""","""04:00""",4.6,"""North Carolina""","""Forsyth"""


In [3]:
df.schema

Schema([('Site Num', Int64),
        ('Date Local', String),
        ('Time Local', String),
        ('Sample Measurement', Float64),
        ('State Name', String),
        ('County Name', String)])

## Be careful of data types on read

If you don't specify things like the data types, character encoding, and value separator character explicitly, Pandas will try to guess. *Sometimes it guesses wrong!*

- **Since the "Site Num" values look like integers but have leading zeros, you need to force Pandas to interpret that column as strings!**
- The date and time columns are interpreted as strings by defult. 
    - If we accept this default, we typically have to convert timestamps into special datetime objects explicitly as a separate step after the read if we want to use the Pandas special date and time functionality. 
    - *We have the option, though, of combining the date and time columns, and parsing them as a datetime object during the CSV read!*

#### *Note on category data type*

Because the State Name, County Name, and Site Num values are repeated many many times throughout the data set, this would be a great opportunity to tell Pandas to interpret them as **"category" data type**. This would **save space in memory** and **make some computations faster**, but introduces a slight complication in the GroupBy functionality since not all categorical combinations appear in the data set. (e.g. not all states have all counties, and not all counties have all site numbers, so when you do the groupby() you need to include `observed=True` as an argument) See the [Groupby_NCcategorical](Groupby_NCcategorical.ipynb) notebook for coverage of those issues. **The category data type really worth using, but I didn't want to complicate the code here for beginners.**

An additional note about using `groupby()` with the category data type is that there is currently a bug in Pandas where if there are NaNs (missing values) in your grouping column, and that grouping column is a categorical type, Pandas won't display the group even if the `dropna=False` flag is set. See [Soner Yıldırım's blog post](https://towardsdatascience.com/be-careful-when-using-pandas-groupby-with-categorical-data-type-a1d31f66b162) for more information and [a link to the release note](https://pandas.pydata.org/docs/whatsnew/v1.5.1.html) from Oct 2022.

### In a single, chained command, we're

- reading the CSV file
    - combining the date + time columns, parsing them as a datetime object, and naming that column "tstamp"
    - forcing the site number to be parsed as a string
    - explicitly specifying the data value separator as a comma
    - explicitly specifying the text character encoding as Unicode 'utf-8'
- renaming four of the columns
- setting the time stamp datetime column "tstamp" as the DataFrame Index

In [4]:
df = pl.read_csv(
    "data/AirDataEPA/NC_NO2_hourly_2018.csv",
    schema_overrides = {'Site Num':'str',
                        'State Name': pl.Categorical,
                        'County Name': pl.Categorical,
                        'Site Num': pl.Categorical},
    separator=',',
    encoding='utf-8'
).with_columns(
    tstamp = (
        pl.col('Date Local') + " " + pl.col('Time Local')
    ).str.strptime(
        pl.Datetime, format='%Y-%m-%d %H:%M'
    )
).drop(
    ['Date Local', 'Time Local']
).rename(
    {
        "State Name": "state",
        "County Name": "county",
        "Site Num": "site",
        "Sample Measurement": "measure"
    }
)

In [5]:
df.head()

site,measure,state,county,tstamp
cat,f64,cat,cat,datetime[μs]
"""0022""",4.4,"""North Carolina""","""Forsyth""",2018-01-01 00:00:00
"""0022""",5.1,"""North Carolina""","""Forsyth""",2018-01-01 01:00:00
"""0022""",3.6,"""North Carolina""","""Forsyth""",2018-01-01 02:00:00
"""0022""",4.1,"""North Carolina""","""Forsyth""",2018-01-01 03:00:00
"""0022""",4.6,"""North Carolina""","""Forsyth""",2018-01-01 04:00:00


## Too much data to see patterns without aggregation

Even in this reduced data set, for a single year there are almost 40k rows!

In [6]:
df.shape

(39205, 5)

---

# GroupBy

## Mean emissions by county

**To start exploring the data and seeing trends or patterns, we need to make comparisons between different counties, sites, dates and times.**

By repeating our basic groupby sMecklenburg County has the highest mean emissions over the whole year for all sites in that county.

In [7]:
df.group_by('county').agg(pl.col('measure').mean())

county,measure
cat,f64
"""Mecklenburg""",8.361926
"""Forsyth""",6.652983
"""Wake""",6.834003


## GroupBy County and Site creates a multi-index

When we group by multiple categorical variables and see that Pandas creates a multi-index. Now we can see that two of the counties have two sites, and their mean emissions vary quite a bit, with the one of the Wake County sites having higher mean emissions than the lower of the two Mecklenburg sites.

In [8]:
(df.group_by(['county','site'])
     .agg(pl.col('measure').mean())
     .sort(pl.col('measure'), descending=True)
)

county,site,measure
cat,cat,f64
"""Mecklenburg""","""0045""",10.198556
"""Wake""","""0021""",8.700441
"""Forsyth""","""0022""",6.652983
"""Mecklenburg""","""0041""",6.28245
"""Wake""","""0014""",4.955026


### Use tuples for multi-index access

To select values from a multi-index, combine the values in a tuple. **Order matters!**

### Partial hierarchy return

Including only the highest level value in a multi-index returns all sub-values

### Can get multi-index in both rows and columns

MultiIndex is a bit of a pain, but that's what you get both in rows and columns when you either do multiple groupby levels or multiple aggregation functions 

In [9]:
(df.group_by(['county','site'])
     .agg(pl.col('measure').min().alias('min'), 
          pl.col('measure').mean().alias('mean'),
          pl.col('measure').max().alias('max')
         )
)

county,site,min,mean,max
cat,cat,f64,f64,f64
"""Wake""","""0014""",0.3,4.955026,38.2
"""Forsyth""","""0022""",0.4,6.652983,41.0
"""Mecklenburg""","""0045""",0.3,10.198556,39.2
"""Wake""","""0021""",0.1,8.700441,35.0
"""Mecklenburg""","""0041""",0.3,6.28245,39.6


### Sort results to see rank order

Sorting values using `.sort_values()` can be a big help in exploration – it gives you a ranking in that measure – but remember:

- select the sorting column(s) **using the tuple notation in a multi-index situation**
- sort_values() **doesn't actually change the underlying DataFrame** – you need to reassign for that

In [10]:
(df.group_by(['county','site'])
     .agg(pl.col('measure').min().alias('min'), 
          pl.col('measure').mean().alias('mean'),
          pl.col('measure').max().alias('max')
         )
     .sort('mean', descending=True)
)

county,site,min,mean,max
cat,cat,f64,f64,f64
"""Mecklenburg""","""0045""",0.3,10.198556,39.2
"""Wake""","""0021""",0.1,8.700441,35.0
"""Forsyth""","""0022""",0.4,6.652983,41.0
"""Mecklenburg""","""0041""",0.3,6.28245,39.6
"""Wake""","""0014""",0.3,4.955026,38.2


---

### Aside: Alternative method chaining line formatting

We're already chaining together multiple DataFrame methods (special name for functions of an object), where the output of one stage is the input for the next. This is good practice so we don't have to create temporary variables at each stage. But, so far we've been breaking lines on parentheses and commas, which is fine with the Python interpreter and puts the arguments for each method at a similar visual level.

**It turns out that the Python interpreter is fine breaking other places if you surround a line of code with parentheses!** It looks strage at first, but it puts each method name at the same visual level, without the closing parentheses before them, so it's slightly easier to see the sequence of the chain of commands. For lots more on this read [Matt Harrison's excellent book Effective Pandas](https://dundermharrison.gumroad.com/l/effectivepandas).

---

# Exploring data time trends

Since our DataFrame index is a datetime, we can use special built-in Pandas date and time functionailty.

## DatetimeIndex Attributes – *categorical discreet date/time parts*

The DatetimeIndex class has a lot of optimized features, which are detailed in [the Index section of the Time series / date functionality documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#indexing)

In Tableau there is the concept of **categorical discreet date parts**, which we use when we want to extract part of a date or time and ignore the rest of the information. For example:

- **Is there a trend in our data through the days of the week?** *Maybe more emissions happen on weekdays, and less on weekends?* In that case we want to know what happens on Sundays, Mondays, etc, but ignore what month or year or hour the data comes from and aggregate over all of those attributes while preserving the weekday categorical distinction.
- **Is there a seasonal trend over the months of the year?** *Maybe there are more emissions during the winter or summer?* In this case all we want to retain is whether the data is from a January, or a February, etc, and ignore (i.e., aggregate over) all other attributes like what hour or day or year they're from.
- **Is there a trend across hours in the day?** *Maybe there are more emissions when more people are awake, or sleeping?* In this case we want to know which of the 24 hours during a day something occurred in, and aggregate across everything else.

[Datetime index attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) – gives you categorical date/time parts like month, day, minute, etc

*Note: One thing I find quite inconvenient is that the date and time attributes turn into integers (starting at 0 or 1, depending on the specifics) instead of staying nice, human-readable values. (e.g., February == 2, Monday == 0, etc.) DatetimeIndex has some methods like `.month_name()`, but by default the pivot tables will be sorted alphabetically rather than in calendar month order...*

## Resampling – *preserves continuous time, but aggregated to a coarser level*

If you want to preserve a sense of continuous time, but aggregate up to a coarser time level than the original data date/time sampling level of detail, there is a concept in Pandas of **resampling** date and time data. Resampling is "a time-based groupby, followed by a reduction method on each of its groups." **For example, if you have hourly data and want to convert that data (through aggregation like a mean or sum) into daily or monthly or yearly data, you need to resample.**

This can be done in conjunction with a groupby() operation. I give one example of this at the very end of this lesson.

- Resample in timeseries documentation: [Date/Time resampling](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling)
- Series.resample() documentation: [Resample method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.resample.html)
- Valid strings for resampling frequency: [date-offset object docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

---

# Pivot table to explore the data

GroupBy is important for creating tidy data that is ready for further analysis, but **sometimes you want to compare across groups visually in a table. In those cases it's easiest to see the patterns in a pivot table, which is a "wide" format, vs the "tall" format of tidy data**. We will also use pivot tables to prepare data for visualization using the Pandas built-in plotting functions.

Note: This is similar to what you would do in Excel – take event data and aggregate by creating a pivot table, and then either view those numbers directly, or create charts from the pivot table.

#### *Pivot table example*

*How the number of data points varies across sites (vertical) and months (horizontal)*

<img src='images/PivotTable_example.png' />

## Pivot table syntax

Pandas pivot_table() documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

```
df.pivot_table(index=None, columns=None, values=None, aggfunc='mean', ...)
```

#### I think of this as specifying how I want to build the eventual table

- **index**: the variable(s) that will become the index *(row labels – vertical)*
- **columns**: the variable(s) that will become the column(s) *(column labels – horizontal)*
- **values**: the column that will be aggregated and become the **body of the table**
- **aggfunc**: the aggregation function that will be used – same as in groupby(). *(A list will generate multiple tables "side-by-side" in the "columns" direction.)*


In [11]:
df.pivot(
    on='site',
    index='county',
    values='measure',
    aggregate_function='sum'
)

county,0022,0041,0045,0014,0021
cat,f64,f64,f64,f64,f64
"""Forsyth""",54873.8,null,null,null,null
"""Mecklenburg""",null,44567.7,81914.8,null,null
"""Wake""",null,null,null,39090.2,69098.9


## Number of records from each site over months of the year

To see how complete our data is, we want to display the count of records over the 12 months of the year for each site. (Note: The grouping over state that I do below is not necessary at all here.) To do this we'll create a pivot table where 

- the rows look like a groupby() combination of state, county and site
- the columns are the extracted discreet datetime Index "month" part
- the values populating the main part of the table are a "count" of the "measure" values

**Some of the counties and sites are more complete than others.** For reference:

- 28 days * 24 hours == 672 samples
- 31 days * 24 hours == 744 samples

### can't pivot directly on the result of an expression

https://github.com/pola-rs/polars/issues/8461

In [12]:
df.with_columns(
    month=pl.col('tstamp').dt.month()
).pivot(
    on='month',
    index=['state','county','site'],
    values='measure',
    aggregate_function='len'
)

state,county,site,1,2,3,4,5,6,7,8,9,10,11,12
cat,cat,cat,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""North Carolina""","""Forsyth""","""0022""",704,631,701,686,697,676,706,691,682,702,669,703
"""North Carolina""","""Mecklenburg""","""0041""",714,644,697,687,705,677,714,704,525,560,467,null
"""North Carolina""","""Mecklenburg""","""0045""",706,635,712,683,703,690,667,713,530,690,614,689
"""North Carolina""","""Wake""","""0014""",705,509,699,678,692,681,692,673,537,636,683,704
"""North Carolina""","""Wake""","""0021""",696,634,696,677,698,681,701,690,537,652,683,597


---

## Output styling with `.style`

It's hard for people to see patterns in large groups of numbers, and sometimes extra visual elements like the ".0" at the end of all of those counts, make it even harder.

**Pandas has a concept of output styling in Jupyter for DataFrames (including pivot tables), similar to Excel's "Conditional Formatting"**

https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

### *Two things to watch out for:*
- Styling requires a unique index, so if you run into an error you may need to `.reset_index()`
- **A styled DataFrame doesn't elide in the way Jupyter typically displays, so it will print out the whole DataFrame below the cell and clog up your window!!** So, don't try it with the original Emissions DataFrame (df) – do something like `.head()` first so the number of rows displayed will be limited.


### Set number precision

Sets the number of decimal places

### Styling in Polars uses Great Tables

https://posit-dev.github.io/great-tables/articles/intro.html

In [13]:
df.with_columns(
    month=pl.col('tstamp').dt.month()
).pivot(
    on='month',
    index=['state','county','site'],
    values='measure',
    aggregate_function='len'
).style

state,county,site,1,2,3,4,5,6,7,8,9,10,11,12
North Carolina,Forsyth,0022,704,631,701,686,697,676,706,691,682,702,669,703
North Carolina,Mecklenburg,0041,714,644,697,687,705,677,714,704,525,560,467,None
North Carolina,Mecklenburg,0045,706,635,712,683,703,690,667,713,530,690,614,689
North Carolina,Wake,0014,705,509,699,678,692,681,692,673,537,636,683,704
North Carolina,Wake,0021,696,634,696,677,698,681,701,690,537,652,683,597


In [14]:
df.with_columns(
    month=pl.col('tstamp').dt.month()
).pivot(
    on='month',
    index=['state','county','site'],
    values='measure',
    aggregate_function='len'
).style.tab_header(
    title="Site measurement counts over months (1-12)", 
    subtitle='Not all sites have the same amount of data'
)

GT(_tbl_data=shape: (5, 15)
┌────────────────┬─────────────┬──────┬─────┬───┬─────┬─────┬─────┬──────┐
│ state          ┆ county      ┆ site ┆ 1   ┆ … ┆ 9   ┆ 10  ┆ 11  ┆ 12   │
│ ---            ┆ ---         ┆ ---  ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ ---  │
│ cat            ┆ cat         ┆ cat  ┆ u32 ┆   ┆ u32 ┆ u32 ┆ u32 ┆ u32  │
╞════════════════╪═════════════╪══════╪═════╪═══╪═════╪═════╪═════╪══════╡
│ North Carolina ┆ Forsyth     ┆ 0022 ┆ 704 ┆ … ┆ 682 ┆ 702 ┆ 669 ┆ 703  │
│ North Carolina ┆ Mecklenburg ┆ 0041 ┆ 714 ┆ … ┆ 525 ┆ 560 ┆ 467 ┆ null │
│ North Carolina ┆ Mecklenburg ┆ 0045 ┆ 706 ┆ … ┆ 530 ┆ 690 ┆ 614 ┆ 689  │
│ North Carolina ┆ Wake        ┆ 0014 ┆ 705 ┆ … ┆ 537 ┆ 636 ┆ 683 ┆ 704  │
│ North Carolina ┆ Wake        ┆ 0021 ┆ 696 ┆ … ┆ 537 ┆ 652 ┆ 683 ┆ 597  │
└────────────────┴─────────────┴──────┴─────┴───┴─────┴─────┴─────┴──────┘, _body=<great_tables._gt_data.Body object at 0x106cdea50>, _boxhead=Boxhead([ColInfo(var='state', type=<ColInfoTypeEnum.default: 1>, column_label='state', column_align='center', column_width=None), ColInfo(var='county', type=<ColInfoTypeEnum.default: 1>, column_label='county', column_align='center', column_width=None), ColInfo(var='site', type=<ColInfoTypeEnum.default: 1>, column_label='site', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='center', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='center', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='center', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='center', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='center', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x15e006f30>, _spanners=Spanners([]), _heading=Heading(title='Site measurement counts over months (1-12)', subtitle='Not all sites have the same amount of data', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x15dfe4290>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=Options

### Background gradient (heatmap) – Higher mean emissions in the winter months

- **Color scale range is determined column-wise by default**. You can change the `axis=1` for row-wise auto-scaling of max and min color values
- The only way to get a common scale across the whole pivot table is to explicitly set `vmin=` and `vmax=`
- You can set a fractional threshold for dark to light text transition with `text_color_threshold`

#### Scaling color independently across rows:

*Allows us to compare across months, but distorts the comparison between sites!*

In [15]:
df.with_columns(
    month=pl.col('tstamp').dt.month()
).pivot(
    on='month',
    index=['state','county','site'],
    values='measure',
    aggregate_function='mean'
).style.data_color(
    columns=cs.numeric(),
    palette=['white','darkblue'],
    domain=[2.8, 15.3],
    na_color='lightgray'
).tab_header(
    title="Site average emissions over months (1-12)", 
    subtitle='We can start to see seasonal trends in the emissions'
).fmt_number(columns=cs.numeric(), decimals=1)

GT(_tbl_data=shape: (5, 15)
┌──────────┬─────────────┬──────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ state    ┆ county      ┆ site ┆ 1         ┆ … ┆ 9        ┆ 10        ┆ 11        ┆ 12        │
│ ---      ┆ ---         ┆ ---  ┆ ---       ┆   ┆ ---      ┆ ---       ┆ ---       ┆ ---       │
│ cat      ┆ cat         ┆ cat  ┆ f64       ┆   ┆ f64      ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪═════════════╪══════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ North    ┆ Forsyth     ┆ 0022 ┆ 10.902699 ┆ … ┆ 4.699853 ┆ 6.278775  ┆ 7.186697  ┆ 10.812233 │
│ Carolina ┆             ┆      ┆           ┆   ┆          ┆           ┆           ┆           │
│ North    ┆ Mecklenburg ┆ 0041 ┆ 13.098319 ┆ … ┆ 4.694286 ┆ 7.169464  ┆ 7.941542  ┆ null      │
│ Carolina ┆             ┆      ┆           ┆   ┆          ┆           ┆           ┆           │
│ North    ┆ Mecklenburg ┆ 0045 ┆ 15.230028 ┆ … ┆ 9.099623 ┆ 10.007681 ┆ 12.466124 ┆ 12.984906 │
│ Carolina ┆             ┆      ┆           ┆   ┆          ┆           ┆           ┆           │
│ North    ┆ Wake        ┆ 0014 ┆ 8.972766  ┆ … ┆ 3.539292 ┆ 6.199686  ┆ 5.627818  ┆ 7.570312  │
│ Carolina ┆             ┆      ┆           ┆   ┆          ┆           ┆           ┆           │
│ North    ┆ Wake        ┆ 0021 ┆ 11.747845 ┆ … ┆ 6.154935 ┆ 8.756595  ┆ 9.185944  ┆ 10.267169 │
│ Carolina ┆             ┆      ┆           ┆   ┆          ┆           ┆           ┆           │
└──────────┴─────────────┴──────┴───────────┴───┴──────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x156af1310>, _boxhead=Boxhead([ColInfo(var='state', type=<ColInfoTypeEnum.default: 1>, column_label='state', column_align='center', column_width=None), ColInfo(var='county', type=<ColInfoTypeEnum.default: 1>, column_label='county', column_align='center', column_width=None), ColInfo(var='site', type=<ColInfoTypeEnum.default: 1>, column_label='site', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='right', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='right', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='right', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='right', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='right', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='right', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='right', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='right', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='right', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='right', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='right', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x15e0279b0>, _spanners=Spanners([]), _heading=Heading(title='Site average emissions over months (1-12)', subtitle='We can start to see seasonal trends in the emissions', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='1', rows=[0]), grpname=None, colname='1', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#5a5ab4')]), StyleInfo(locname=LocBody(col

#### Common color scale across whole table

**Lets us in an undistorted way compare not only across months, but across sites.** Notice how this method chaining line formatting makes the long string of commands easier to read.

---

**To try the exercise below, select this cell and from the Jupyter menus choose**

`Run -> Run All Above Selected Cell`

## EXERCISE

**Explore with a heatmap-styled pivot table, how mean emissions at each site vary by hour of the day**

Reminder: [Datetime index attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) – gives you categorical date/time parts like month, day, minute, etc

*Note: Type instead of using copy/paste for better retention*

*Expected output*

<img src='images/Emissions_heatmap_hourofday.png' />

---

### Horizontal Bar styling

Again, this is like one of the conditional formatting options for tables in Excel

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.bar.html

- Notice that the default is `vmin=0` so you don't distort perception of the data, so you don't have to type that explicitly. That's great and hasn't always been the default (zero bar length at the minimum value used to be the default, which was bad, perceptually).
- **But, you do need to explicily set `vmax` to get a common maximum bar length across all columns!**

#### Higher median in the winter

*Notice how we just swapped what is the index and what is the columns*

### Groupby results can be styled, too

*Just make sure the DataFrame isn't too long, or you choose with head() or tail() to only display part of it, or it will overwhelm your browser window.*

In [16]:
(df.group_by(['county','site'])
     .agg(pl.col('measure').mean())
     .sort(pl.col('measure'), descending=True)
     .style
     .fmt_number(columns='measure', decimals=1)
     .data_color(
        columns='measure',
        palette=['white','darkblue'],
        )
)

county,site,measure
Mecklenburg,0045,10.2
Wake,0021,8.7
Forsyth,0022,6.7
Mecklenburg,0041,6.3
Wake,0014,5.0


---

# Time series plots like Excel

The pivot tables themselves provide a visualization of the data, but **a line plot is a very natural visualization style for a time series.**

To use the built-in plotting functions of Pandas for data like this, just like in Excel we need to be careful which data goes in which direction in the pivot table.

- **Intended X-axis needs to be on the row index** – so, we need time to be going down the row index if we want time to be horizontal, from left to right, in our line plot
- **Separate lines will be made from each column** – so we want our counties and sites to be our columns

*This happens to be the same arrangement we made for our horizontal bars styling above.*

#### Pandas line plot documentation

- General plot: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html
- Line plot: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.line.html

### Higher emissions variance in winter months

In [17]:
df.with_columns(
    month = pl.col('tstamp').dt.month()
).group_by(
    ['county','site','month']
).agg(
    pl.col('measure').var()
).plot.line(
    x = 'month:O',
    y = 'measure:Q',
    color = 'county_site:N'
).transform_calculate(
    county_site = 'datum.county + " " + datum.site'
).properties(
    width=400
)

alt.Chart(...)

In [18]:
df.with_columns(
    month = pl.col('tstamp').dt.month(),
    county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' ')
).group_by(
    ['county_site','month']
).agg(
    pl.col('measure').var()
).plot.line(
    x = 'month:O',
    y = 'measure:Q',
    color = 'county_site:N'
).properties(
    width=400
)

alt.Chart(...)

#### Emissions are lower on the weekends

- We'll rotate the x-axis labels by 90 degrees so they don't run together. This isn't great for reading...
- We combine the day of the week integer and the day name as a quick workaround to order the days properly (since the default is to order alphabetically)

### Day of week conversion

https://github.com/pola-rs/polars/issues/13554

- `.dt.to_string()` is an alias for `.dt.strftime()` so they're interchangeable
- `%A` is spelled out and `%a` is three letter abbreviation
- Altair loses the categorical ordering from Pandas or Polars, so have to specify again in the `.sort()`

In [19]:
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

df.with_columns(
    county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' '),
    # weekday = pl.col('tstamp').dt.to_string('%a').cast(pl.Enum(day_order))
    weekday = pl.col('tstamp').dt.to_string('%a')
).group_by(
    ['county_site','weekday']
).agg(
    pl.col('measure').mean()
).plot.line(
    x = alt.X('weekday:O').sort(day_order),
    y = 'measure:Q',
    color = 'county_site:N'
).properties(
    width=400
)

alt.Chart(...)

---

**To try the exercise below, select this cell and from the Jupyter menus choose**

`Run -> Run All Above Selected Cell`

## EXERCISE

**Produce a line plot of mean emissions by hour of day, with a line for each site**

*Note: Type instead of using copy/paste for better retention*

*Expected output*

<img src='images/LinePlot_hourofday.png' />

---

## Select certain columns to plot & figure size

- You can use the `y=` argument to select certain columns to plot
- **Notice how we have to use a tuple here to select the column(s)!**
- and the figure size with `figsize=`

*Note that `df.index.week` has been deprecated, so we need to use `df.index.isocalendar().week` to get weekly aggregation*

### Subplots for one line per plot

- Use the `subplots=True` argument to make small multiples if the plots overlap too much
- The `figsize` is for the combination of all subplots (not the size of each individual)
- I'm using a combination of (year, month, day) instead of index.day_of_year for readability

---

**To try the exercise below, select this cell and from the Jupyter menus choose**

`Run -> Run All Above Selected Cell`

## EXERCISE

**Produce subplots, one for each county and site, showing the fully-detailed hourly data**

*Note: Type instead of using copy/paste for better retention*

Expected output:

<img src='images/LineSubPlots_hourly.png' />

---

# Resample for continuous time plots

Here's an example of resampling to the "month start" `('MS')` level in time while aggregating with both the mean and minimum.

- Valid strings for resampling frequency: [date-offset object docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)
- General Resample documentation: [Resample docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling) *(I have some trouble understanding this...)*

Because the groupby object still contains the same rows as the original, with the same index, you can apply `.resample()` after the `.groupby()`, which resamples within each group before the aggregation operations.

### group_by_dynamic requires sorted dates/times in Polars

https://docs.pola.rs/user-guide/transformations/time-series/rolling/

Possible time intervals/spans: https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.group_by_dynamic.html

In [20]:
df.sort(
    'tstamp'
).group_by_dynamic(
    'tstamp', 
    every='1mo',
    group_by=['county','site']
).agg(
    pl.col('measure').mean()
)

county,site,tstamp,measure
cat,cat,datetime[μs],f64
"""Forsyth""","""0022""",2018-01-01 00:00:00,10.902699
"""Forsyth""","""0022""",2018-02-01 00:00:00,7.871474
"""Forsyth""","""0022""",2018-03-01 00:00:00,6.535093
"""Forsyth""","""0022""",2018-04-01 00:00:00,4.920262
"""Forsyth""","""0022""",2018-05-01 00:00:00,5.289096
…,…,…,…
"""Wake""","""0021""",2018-08-01 00:00:00,7.851304
"""Wake""","""0021""",2018-09-01 00:00:00,6.154935
"""Wake""","""0021""",2018-10-01 00:00:00,8.756595


In [21]:
df.sort(
    'tstamp'
).group_by_dynamic(
    'tstamp', 
    every='2w',
    group_by=['county','site']
).agg(
    pl.col('measure').mean()
).with_columns(
    county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' '),
).plot.line(
    x = alt.X('tstamp:T'),
    y = 'measure:Q',
    color = 'county_site:N'
).properties(
    width=500
)

alt.Chart(...)

In [22]:
df.sort(
    'tstamp'
).group_by_dynamic(
    'tstamp', 
    every='2w',
    group_by=['county','site']
).agg(
    pl.col('measure').mean()
).with_columns(
    county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' '),
).plot.line(
    x = alt.X('tstamp:T'),
    y = 'measure:Q',
    color = 'county_site:N'
).properties(
    width=100,
    height=100
).facet(
    column='county_site'
)

alt.FacetChart(...)

### Pandas-native plotting still requires a pivot table

The only way I know how to arrange the data for Pandas-style plotting is to 

- do the `.groupby()`
- do the `.resample()` on the groups – here at the day `('D')` level
- `.reset_index()` to get the data ready for a pivot_table by moving the multi-index back into normal columns
- create the `.pivot_table()` like above
- `.plot()`

---

# Plotting tidy data right out of groupby()

I like Altair, Seaborn, and Plotly for plotting in Python because they want tidy data that I can get right from a `groupby()` result without having to create a pivot table. That way I don't have to worry about which axis is which in my Pandas DataFrame – I can assign any variable to either plot axis.

- No need to create a pivot table – work directly with groupby()
- Capabilities for creating faceted plots
- Seaborn is better for built-in complicated statistical plots, but has hard-to-remember names for the plotting functions
- Altair has a more consistent interface for all types of plots, and it has built-in aggregation, including time-series, but for large data it works out better to filter and group first rather than letting Altair do the work
- Plotly has nice interaction built in, but Jupyter extension requires setup

**Note: You almost always have to run .reset_index() on the grouped, agregated DataFrame before you can plot with these modules.**

## Faceted line plot with Seaborn

`relplot()` is the Seaborn "Figure-level interface for drawing relational plots onto a FacetGrid"

- https://seaborn.pydata.org/examples/faceted_lineplot.html
- https://seaborn.pydata.org/generated/seaborn.relplot.html

*If you don't specify a categorical color palette, Seaborn will look at the values in "site" and re-interpret them as integers, then use a sequential (numerical) palette!!*

- https://seaborn.pydata.org/generated/seaborn.color_palette.html

**You need to use `.reset_index()` to access the index column(s) to plot.**

---

## Altair for data visualization

Altair is a very nice alternative module for visualizing data that's in tidy form, right out of the aggregated groupby results. Another nice feature is that Altair has a lot of built-in aggregation itself, as well as filtering, calculations, and interaction (including linking multiple plots), so you have extra options for date/time funcitons and nice display.

You can see examples of this in the [Groupby_NCcategorical notebook](Groupby_NCcategorical.ipynb). 
More in-depth instruction can be found in 
[the video of my Altair workshop](https://library.capture.duke.edu/Panopto/Pages/Viewer.aspx?id=df15cf8e-86e1-4c6f-ac16-aab600f06c48) 
and the 
[accompanying GitHub repository](https://github.com/emonson/altair-vis-python)

*Example Altair output:*

<img src='images/Altair_monthdaysite.svg' />

In [25]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [28]:
alt.Chart(
    df.with_columns(
        county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' ')
    )
).mark_line(clip=True).encode(
    x='hours(tstamp):T',
    y='mean(measure):Q',
    color='county_site:N',
    detail='county_site:N',
    tooltip='site:N'
).properties(
    width=90,
    height=90
).facet(
    facet='yearmonth(tstamp):T',
    columns=6
)

alt.FacetChart(...)

### Polars categorical/Enum ordering

In [30]:
alt.data_transformers.enable("default")

DataTransformerRegistry.enable('default')

### Chrono strftime documentaion

https://docs.rs/chrono/latest/chrono/format/strftime/index.html

- If you use the Altair built-in date time stuff it sorts automatically, plus has options for the monthyear() sorts of things
- Looks like it works okay if you carry along some extra columns for sorting, but it's more of a pain

In [72]:
df.with_columns(
    county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' '),
    month_year = pl.concat_str(
        [pl.col('tstamp').dt.to_string('%b'), 
         pl.col('tstamp').dt.to_string('%Y')],
         separator=' '),
    year_month = pl.concat_str(
        [pl.col('tstamp').dt.to_string('%Y'), 
         pl.col('tstamp').dt.to_string('%m')],
         separator=''),
    hour = pl.col('tstamp').dt.hour()
).group_by(
    ['county_site', 'month_year', 'year_month', 'hour']
).agg(
    pl.col('measure').mean().alias('mean_measure')
)

county_site,month_year,year_month,hour,mean_measure
str,str,str,i8,f64
"""Mecklenburg 0045""","""May 2018""","""201805""",12,5.266667
"""Wake 0021""","""Oct 2018""","""201810""",20,11.667857
"""Wake 0014""","""May 2018""","""201805""",17,2.803333
"""Mecklenburg 0041""","""Jul 2018""","""201807""",20,5.764516
"""Wake 0014""","""Aug 2018""","""201808""",19,6.526667
…,…,…,…,…
"""Wake 0021""","""Dec 2018""","""201812""",16,10.12963
"""Wake 0021""","""Nov 2018""","""201811""",19,11.21
"""Wake 0021""","""Oct 2018""","""201810""",8,10.855556


In [71]:
alt.Chart(
    df.with_columns(
        county_site = pl.concat_str([pl.col('county'), pl.col('site')], separator=' '),
        month_year = pl.concat_str(
            [pl.col('tstamp').dt.to_string('%b'), 
             pl.col('tstamp').dt.to_string('%Y')],
             separator=' '),
        year_month = pl.concat_str(
            [pl.col('tstamp').dt.to_string('%Y'), 
             pl.col('tstamp').dt.to_string('%m')],
             separator=''),
        hour = pl.col('tstamp').dt.hour()
    ).group_by(
        ['county_site', 'month_year', 'year_month', 'hour']
    ).agg(
        pl.col('measure').mean().alias('mean_measure')
    )
).mark_line(clip=True).encode(
    x=alt.X('hour:Q'),
    y=alt.Y('mean_measure:Q'),
    color='county_site:N',
    detail='county_site:N',
    tooltip='county_site:N'
).properties(
    width=90,
    height=90
).facet(
    facet=alt.Facet('month_year:O').sort(field='year_month'),
    columns=6
)

alt.FacetChart(...)

---

## Aside: Ordering the days of the week

Above I used a trick to sort the days of the week names in a natural order rather than alphabetically. If we want to do this in a more principled way we can use ordered an ordered categorical data type.

### Method 1

After we create the pivot table we can replace the index with an ordered CategoricalIndex and then sort that index before plotting

### Method 2

We can make the day_name() output an ordered categorical data type directly within the pivot table command, but **we then need to include the `observed=True` argument** or for some reason we'll end up with all combinations of the counties and sites (even though the counties and sites aren't categorical)...